In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Preproccessing and indexing data into Elastic Search

In [ ]:
import csv
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# Connect to Elasticsearch
es = Elasticsearch(
    cloud_id="IR_Assignment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ3YzUzMzQxY2QwZWE0ODFmOTVhMGNhMTY1OGJjMmU4NiQ0OWFmMjNhMTA1MGM0NTgxYjNlMTc5ZThmMGFjOGY5Mg==",
    basic_auth=("elastic", "21m15STAOB1AQEBZq3wkBedm")
)

# Read the CSV file and create a list of game dictionaries to be bulk indexed into Elasticsearch
games = []
with open("/content/drive/MyDrive/Colab Files/Video_Games_Sales_as_at_22_Dec_2016.csv") as f:
    reader = csv.DictReader(f)
    for row in reader:
        # Clean up the data by filling NaN values with 0
        for key in row.keys():
            if row[key] == "N/A":
                row[key] = 0
        # Convert the numerical columns to float or integer
        # row["Rank"] = int(row["Rank"])
        row["Year_of_Release"] = int(row["Year_of_Release"]) if row["Year_of_Release"] != 0 else 0
        row["NA_Sales"] = float(row["NA_Sales"])
        row["EU_Sales"] = float(row["EU_Sales"])
        row["JP_Sales"] = float(row["JP_Sales"])
        row["Other_Sales"] = float(row["Other_Sales"])
        row["Global_Sales"] = float(row["Global_Sales"])
        row["Critic_Score"] = float(row["Critic_Score"]) if row["Critic_Score"] and row["Critic_Score"] != '0' else None
        row["Critic_Count"] = int(row["Critic_Count"]) if row["Critic_Count"] and row["Critic_Count"] != "N/A" else None
        row["User_Score"] = float(row["User_Score"]) if row["User_Score"] != "tbd" and row["User_Score"] and row["User_Score"] != '0' else None
        row["User_Count"] = int(row["User_Count"]) if row["User_Count"] != '' else None
        games.append(row)

# Define the index mapping
index_mappings = {
    "mappings": {
        "properties": {
            # "Rank": {"type": "integer"},
            "Name": {"type": "text"},
            "Platform": {"type": "keyword"},
            "Year_of_Release": {"type": "integer"},
            "Genre": {"type": "keyword"},
            "Publisher": {"type": "keyword"},
            "NA_Sales": {"type": "float"},
            "EU_Sales": {"type": "float"},
            "JP_Sales": {"type": "float"},
            "Other_Sales": {"type": "float"},
            "Global_Sales": {"type": "float"},
            "Critic_Score": {"type": "float"},
            "Critic_Count": {"type": "integer"},
            "User_Score": {"type": "float"},
            "User_Count": {"type": "integer"},
            "Developer": {"type": "keyword"},
            "Rating": {"type": "keyword"}
        }
    }
}

# Delete the index if it already exists
if es.indices.exists(index="video_games"):
    es.indices.delete(index="video_games")

# Create the index
es.indices.create(index="video_games", body=index_mappings)

# Index the data
bulk_data = []
for game in games:
    bulk_data.append({
        "_index": "video_games",
        "_source": game
    })
    if len(bulk_data) == 1000:
        bulk(es, bulk_data)
        bulk_data = []
if bulk_data:
    bulk(es, bulk_data)


<ipython-input-3-28ad3a9d6207>:65: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index="video_games", body=index_mappings)


In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

# Connect to Elasticsearch
es = Elasticsearch(
    cloud_id="IR_Assignment:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQ3YzUzMzQxY2QwZWE0ODFmOTVhMGNhMTY1OGJjMmU4NiQ0OWFmMjNhMTA1MGM0NTgxYjNlMTc5ZThmMGFjOGY5Mg==",
    basic_auth=("elastic", "21m15STAOB1AQEBZq3wkBedm")
)
res = es.indices.get(index="video_games")
print(res)


{'video_games': {'aliases': {}, 'mappings': {'properties': {'Critic_Count': {'type': 'integer'}, 'Critic_Score': {'type': 'float'}, 'Developer': {'type': 'keyword'}, 'EU_Sales': {'type': 'float'}, 'Genre': {'type': 'keyword'}, 'Global_Sales': {'type': 'float'}, 'JP_Sales': {'type': 'float'}, 'NA_Sales': {'type': 'float'}, 'Name': {'type': 'text'}, 'Other_Sales': {'type': 'float'}, 'Platform': {'type': 'keyword'}, 'Publisher': {'type': 'keyword'}, 'Rating': {'type': 'keyword'}, 'User_Count': {'type': 'integer'}, 'User_Score': {'type': 'float'}, 'Year_of_Release': {'type': 'integer'}}}, 'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'video_games', 'creation_date': '1680018449183', 'number_of_replicas': '1', 'uuid': 'ylVzhsvkSkaOKEi-fw2DRw', 'version': {'created': '8060299'}}}}}


In [ ]:
mapping = {
    "properties": {
        "Global_Sales": {
            "type": "float"
        }
    }
}

# Create the mapping for the "video_games" index
es.indices.put_mapping(index="video_games", body=mapping)


<ipython-input-5-b5946a603c61>:10: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.put_mapping(index="video_games", body=mapping)


ObjectApiResponse({'acknowledged': True})

#Query 1,2,3

In [ ]:
import csv
# 1. Which game has the highest global sales? ?
# Execute query
res = es.search(index="video_games", body={
  "query": {
    "match_all": {}
  },
  "sort": {
    "Global_Sales": {
      "order": "desc"
    }
  },
  "size": 1
})

best_selling_game = res["hits"]["hits"][0]["_source"]["Name"]


print("query_answer: ",res)
print()
print("1. Which game has the highest global sales? ?")
print()
print("Best Selling Game: ",best_selling_game)
print()


best_selling_genre = res["hits"]["hits"][0]["_source"]["Genre"]
print("2. Which is the best selling genre overall ?")
print()
print("Best Selling Genre: ",best_selling_genre)
print()

publisher_with_highest_sales = res["hits"]["hits"][0]["_source"]["Publisher"]
print("3. Which publisher has the highest number of global sales ?")
print()
print("publisher with highest sales:",publisher_with_highest_sales)
print()

query_answer:  {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': [{'_index': 'video_games', '_id': 'kbznKIcBFUMk-Q6dI1vh', '_score': None, '_source': {'Name': 'Wii Sports', 'Platform': 'Wii', 'Year_of_Release': 2006, 'Genre': 'Sports', 'Publisher': 'Nintendo', 'NA_Sales': 41.36, 'EU_Sales': 28.96, 'JP_Sales': 3.77, 'Other_Sales': 8.45, 'Global_Sales': 82.53, 'Critic_Score': 76.0, 'Critic_Count': 51, 'User_Score': 8.0, 'User_Count': 322, 'Developer': 'Nintendo', 'Rating': 'E'}, 'sort': [82.53]}]}}

1. Which game has the highest global sales? ?

Best Selling Game:  Wii Sports

2. Which is the best selling genre overall ?

Best Selling Genre:  Sports

3. Which publisher has the highest number of global sales ?

publisher with highest sales: Nintendo



<ipython-input-6-4cd21933e0ae>:4: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="video_games", body={


#Query 4

In [ ]:
# Query to find the average sales in North America and global average sales
query = {
    "size": 0,
    "aggs": {
        "sales": {
            "stats": {
                "field": "NA_Sales"
            }
        },
        "global_sales": {
            "stats": {
                "field": "Global_Sales"
            }
        }
    }
}

# Execute query
result = es.search(index="video_games", body=query)

print("4. What is the typical sales volume in North America? What does this imply for the average global sales?")
print()

# Get the worldwide and North American average sales.
average_na_sales = result["aggregations"]["sales"]['avg']
average_global_sales = result["aggregations"]["global_sales"]['avg']

print("average na sales:",average_na_sales,"average global sales:",average_global_sales)
print()

# Compare the two averages
if average_na_sales > average_global_sales:
    print("The average sales in North America are higher than those globally.")
else:
    print("The average sales in North America are lower than those elsewhere.")
print()


<ipython-input-7-cf507df2dc7d>:19: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="video_games", body=query)


4. What is the typical sales volume in North America? What does this imply for the average global sales?

average na sales: 0.3133199999630451 average global sales: 0.6329242856989482

The average sales in North America are lower than those elsewhere.



#Query 5

In [ ]:
# Query to find top 5 games
query_top_games = {
    "size": 5,
    "sort": [
        {"Global_Sales": {"order": "desc"}}
    ],
    "query": {
        "match_all": {}
    }
}
# Query to find top 5 genres for each game
query_top_genres = {
    "size": 0,
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "Platform": ""
                    }
                }
            ]
        }
    },
    "aggs": {
        "top_genres": {
            "terms": {
                "field": "Genre",
                "size": 5,
                "order": {
                    "max_sales": "desc"
                }
            },
            "aggs": {
                "max_sales": {
                    "max": {
                        "field": "Global_Sales"
                    }
                }
            }
        }
    }
}

# Query to find top 5 games
res = es.search(index="video_games", body=query_top_games)
print()
print("5. Identify the top 5 video games by global sales. For each of them, what are the top 5 genres?")
print()
print(res)
print()
# Find top 5 genres for each game
for game in res["hits"]["hits"]:
    # Update query with game name
    query_top_genres["query"]["bool"]["must"][0]["match"]["Platform"] = game["_source"]["Platform"]
    # Execute query
    res_genres = es.search(index="video_games", body=query_top_genres)
    print("genres_result of ",game["_source"]["Name"],res_genres)
    print()
    print("The top 5 genres of",game["_source"]["Name"],"of",game["_source"]["Platform"],"Platform")
    for i in res_genres["aggregations"]['top_genres']["buckets"]:
      print(i["key"])
    print()


<ipython-input-8-10746dea8092>:46: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="video_games", body=query_top_games)



5. Identify the top 5 video games by global sales. For each of them, what are the top 5 genres?

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': None, 'hits': [{'_index': 'video_games', '_id': 'kbznKIcBFUMk-Q6dI1vh', '_score': None, '_source': {'Name': 'Wii Sports', 'Platform': 'Wii', 'Year_of_Release': 2006, 'Genre': 'Sports', 'Publisher': 'Nintendo', 'NA_Sales': 41.36, 'EU_Sales': 28.96, 'JP_Sales': 3.77, 'Other_Sales': 8.45, 'Global_Sales': 82.53, 'Critic_Score': 76.0, 'Critic_Count': 51, 'User_Score': 8.0, 'User_Count': 322, 'Developer': 'Nintendo', 'Rating': 'E'}, 'sort': [82.53]}, {'_index': 'video_games', '_id': 'krznKIcBFUMk-Q6dI1vh', '_score': None, '_source': {'Name': 'Super Mario Bros.', 'Platform': 'NES', 'Year_of_Release': 1985, 'Genre': 'Platform', 'Publisher': 'Nintendo', 'NA_Sales': 29.08, 'EU_Sales': 3.58, 'JP_Sales': 6.81, 'Other_Sales': 0.77, 'Glo

<ipython-input-8-10746dea8092>:57: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res_genres = es.search(index="video_games", body=query_top_genres)


genres_result of  Wii Sports {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1320, 'relation': 'eq'}, 'max_score': None, 'hits': []}, 'aggregations': {'top_genres': {'doc_count_error_upper_bound': 0, 'sum_other_doc_count': 587, 'buckets': [{'key': 'Sports', 'doc_count': 260, 'max_sales': {'value': 82.52999877929688}}, {'key': 'Racing', 'doc_count': 93, 'max_sales': {'value': 35.52000045776367}}, {'key': 'Misc', 'doc_count': 280, 'max_sales': {'value': 28.920000076293945}}, {'key': 'Platform', 'doc_count': 58, 'max_sales': {'value': 28.31999969482422}}, {'key': 'Fighting', 'doc_count': 42, 'max_sales': {'value': 12.84000015258789}}]}}}

The top 5 genres of Wii Sports of Wii Platform
Sports
Racing
Misc
Platform
Fighting

genres_result of  Super Mario Bros. {'took': 1, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 98, 'relation': 'eq'}, 'm

# Query 6

In [ ]:
# Get all games that are "Action" or "Fighting"-classified using a query.
# To get all pertinent information, I set the size to the maximum allowed (10,000 hits).
action_fighting_query = {
    "size": 10000,
    "query": {
        "bool": {
            "should": [
                {"match": {"Genre": "Action"}},
                {"match": {"Genre": "Fighting"}}
            ]
        }
    }
}

# Execute the query and get the results
action_fighting_res = es.search(index="video_games", body=action_fighting_query)
print()
print("6. Find all the games which can be categorized as “Action” or “Fighting”")
print()
print("Query result:", action_fighting_res)
print()

# Extract the game names, then add them to a list.
action_fighting_names = [hit["_source"]["Name"] for hit in action_fighting_res["hits"]["hits"]]

# Print the names of the games
print(action_fighting_names)


<ipython-input-9-6852d97be8dc>:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  action_fighting_res = es.search(index="video_games", body=action_fighting_query)



6. Find all the games which can be categorized as “Action” or “Fighting”

Query result: {'took': 61, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4219, 'relation': 'eq'}, 'max_score': 2.9797127, 'hits': [{'_index': 'video_games', '_id': 'uLznKIcBFUMk-Q6dI1vh', '_score': 2.9797127, '_source': {'Name': 'Super Smash Bros. Brawl', 'Platform': 'Wii', 'Year_of_Release': 2008, 'Genre': 'Fighting', 'Publisher': 'Nintendo', 'NA_Sales': 6.62, 'EU_Sales': 2.55, 'JP_Sales': 2.66, 'Other_Sales': 1.01, 'Global_Sales': 12.84, 'Critic_Score': 93.0, 'Critic_Count': 81, 'User_Score': 8.9, 'User_Count': 1662, 'Developer': 'Game Arts', 'Rating': 'T'}}, {'_index': 'video_games', '_id': '8bznKIcBFUMk-Q6dI1vh', '_score': 2.9797127, '_source': {'Name': 'Super Smash Bros. for Wii U and 3DS', 'Platform': '3DS', 'Year_of_Release': 2014, 'Genre': 'Fighting', 'Publisher': 'Nintendo', 'NA_Sales': 3.27, 'EU_Sales': 1.37, 'JP_Sales': 2.43, 'Othe